In [ ]:
import os
import urllib
import requests
import re
from bs4 import BeautifulSoup
import time

In [ ]:
def get_image(url, count, path):
    '''从原图url中将原图保存到本地'''
    try:
        time.sleep(0.5)
        urllib.request.urlretrieve(url, f'{path}/result{count + 1}.jpg')
    except Exception as e:
        time.sleep(1)
        print("本张图片获取异常，跳过...")
    else:
        print(f"图片+1，成功保存 {count + 1} 张图")

In [ ]:
def find_img_url_from_html(html, rule, count, path):
    soup = BeautifulSoup(html, "lxml")
    link_list = soup.find_all("a", class_="image-link")  # 根据不同网站的HTML结构调整选择器
    for link in link_list:
        result = re.search(rule, str(link))
        if result:
            url = result.group(0)[8:]
            get_image(url, count, path)
            count += 1
    return count

def get_start_html(url, key, first, load_num, sfx):
    page = urllib.request.Request(url.format(key, first, load_num, sfx), headers={'Connection': 'close'})
    html = urllib.request.urlopen(page)
    return html

def image_crawler(search_query, count_num=20, load_num=35, site="bing"):
    site == "bing":
    url = "https://cn.bing.com/images/search?q={}"
    rule = re.compile(r"\"murl\"\:\"http\S[^\"]+")
   

    key = urllib.parse.quote(search_query)
    first = 1
    sfx = 1
    count = 0

    total_count = count_num

In [ ]:
    path_train = f'./imgs/{search_query}/train'
    os.makedirs(path_train, exist_ok=True)
    path_valid = f'./imgs/{search_query}/valid'
    os.makedirs(path_valid, exist_ok=True)
    path_test = f'./imgs/{search_query}/test'
    os.makedirs(path_test, exist_ok=True)

    group_sizes = {'train': 0, 'valid': 0, 'test': 0}
    group_counts = {'train': 0, 'valid': 0, 'test': 0}

    group_sizes['train'] = int(input("请输入 train 组的图片数量："))
    group_sizes['valid'] = int(input("请输入 valid 组的图片数量："))
    group_sizes['test'] = int(input("请输入 test 组的图片数量："))

    while count < total_count:
        html = get_start_html(url, key, first, load_num, sfx)
        count = find_img_url_from_html(html, rule, count, path_train)
        first = count + 1
        sfx += 1

        group_counts['train'] += 1
        if group_counts['train'] % group_sizes['train'] == 0:
            path_train = f'./imgs/{search_query}/train/group_{group_counts["train"] // group_sizes["train"]}'
            os.makedirs(path_train, exist_ok=True)

        if count >= total_count:
            break

        html = get_start_html(url, key, first, load_num, sfx)
        count = find_img_url_from_html(html, rule, count, path_valid)
        first = count + 1
        sfx += 1

        group_counts['valid'] += 1
        if group_counts['valid'] % group_sizes['valid'] == 0:
            path_valid = f'./imgs/{search_query}/valid/group_{group_counts["valid"] // group_sizes["valid"]}'
            os.makedirs(path_valid, exist_ok=True)

        if count >= total_count:
            break

        html = get_start_html(url, key, first, load_num, sfx)
        count = find_img_url_from_html(html, rule, count, path_test)
        first = count + 1
        sfx += 1

        group_counts['test'] += 1
        if group_counts['test'] % group_sizes['test'] == 0:
            path_test = f'./imgs/{search_query}/test/group_{group_counts["test"] // group_sizes["test"]}'
            os.makedirs(path_test, exist_ok=True)


In [ ]:
def main():
    search_query = input("请输入搜索关键词：")
    count_num = int(input("请输入要爬取的图片数量："))
    load_num = int(input("请输入加载的图片数量："))
    image_crawler(search_query, count_num, load_num, site)

if __name__ == "__main__":
    main()